In [9]:
import pandas as pd
import requests
from datetime import datetime
import json
import os
from dateutil import parser
import collections

The following code helps to determine whether new data is inserted in api database or not, if so then we have to pull new movies data and put that in database and update tracking parameters. 

In [2]:
last_page =  requests.get('https://yts.torrentbay.to/api/v2/list_movies.json?page=821&limit=50').json()
first_page = requests.get('https://yts.torrentbay.to/api/v2/list_movies.json?page=1&limit=50').json()

In [3]:
first_page

{'status': 'ok',
 'status_message': 'Query was successful',
 'data': {'movie_count': 42182,
  'limit': 50,
  'page_number': 1,
  'movies': [{'id': 42880,
    'url': 'https://yts.torrentbay.to/movies/comedy-chingonas-2021',
    'imdb_code': 'tt16152220',
    'title': 'Comedy Chingonas',
    'title_english': 'Comedy Chingonas',
    'title_long': 'Comedy Chingonas (2021)',
    'slug': 'comedy-chingonas-2021',
    'year': 2021,
    'rating': 5.7,
    'runtime': 0,
    'genres': ['Comedy'],
    'summary': 'A group of hilarious Latina comics come together for an hour of no-holds-barred stand-up hosted by Danny Trejo and Izabella Alvarez. Filmed live at the HA Comedy Festival in San Antonio, the dynamic roster of talent includes sketch comedy writer and stand-up Vanessa Gonzalez; self-proclaimed &quot;mommy comic&quot; Debi Gutierrez; transgender model, activist, showgirl, and CEO Carmen Carrera; New York-based comedian and actress Xazmin Garza; MadTV alum Jill-Michele Meleán; lesbian comic C

In [6]:
last_uploaded_date = parser.parse('2022-04-28')

In [7]:
count = 0
for movie in first_page['data']['movies']:
    if last_uploaded_date < parser.parse(movie['torrents'][0]['date_uploaded'].split(' ')[0]):
        count +=1
        print(movie['title'])

Love, Life and Goldfish
Monkey Enters Lanka
Siembra
Les Rose
Disorientation Is Not a Crime
Snipers
Accused of Murder
Brad Williams: Fun Size
Honokaa Boy
American Friends
Collective: Unconscious
Adventures of a Private Eye
Trophy Wife
Due Season
Circus of the Dead
Vietnam, Texas
Bright Road
I Don't Have Any Money Left in My Retirement Account
The Killing Game
Prayers for the Stolen
Going to Hell: The Movie
Prelude to a Kiss
The Heroin Busters
The Big Racket
Honeymoon with My Mother
Rumspringa
The Aviary
Marvelous and the Black Hole
Crush
Unplugging
White Rhino
In the Wake
BBC Sunday-Night Theatre Nineteen Eighty-Four
Dr. Strange
Look in Any Window


In [8]:
count

35

In [4]:
last_page

{'status': 'ok',
 'status_message': 'Query was successful',
 'data': {'movie_count': 42163,
  'limit': 50,
  'page_number': 821,
  'movies': [{'id': 1168,
    'url': 'https://yts.torrentbay.to/movies/firewalker-1986',
    'imdb_code': 'tt0091055',
    'title': 'Firewalker',
    'title_english': 'Firewalker',
    'title_long': 'Firewalker (1986)',
    'slug': 'firewalker-1986',
    'year': 1986,
    'rating': 5.1,
    'runtime': 105,
    'genres': ['Action', 'Adventure', 'Comedy'],
    'summary': 'Max Donigan sets off on a treasure hunt with his friend Leo and newly-acquired partner Patricia, who provided the treasure map. Along the way they encounter a few bar fights, evil &quot;coyotes&quot;, and other obstacles they are able to overcome with ease. When they find the temple with the treasure, they also find the Firewalker, who wants to retain the power of his ancestors and put an end to the trio&#39;s treasure hunt. In the end, Max and his two friends persevere and return home wealthy

In [13]:
last_page =  requests.get('https://yts.torrentbay.to/api/v2/list_movies.json?page=821&limit=50').json()
last_page

{'status': 'ok',
 'status_message': 'Query was successful',
 'data': {'movie_count': 42163,
  'limit': 50,
  'page_number': 821,
  'movies': [{'id': 1168,
    'url': 'https://yts.torrentbay.to/movies/firewalker-1986',
    'imdb_code': 'tt0091055',
    'title': 'Firewalker',
    'title_english': 'Firewalker',
    'title_long': 'Firewalker (1986)',
    'slug': 'firewalker-1986',
    'year': 1986,
    'rating': 5.1,
    'runtime': 105,
    'genres': ['Action', 'Adventure', 'Comedy'],
    'summary': 'Max Donigan sets off on a treasure hunt with his friend Leo and newly-acquired partner Patricia, who provided the treasure map. Along the way they encounter a few bar fights, evil &quot;coyotes&quot;, and other obstacles they are able to overcome with ease. When they find the temple with the treasure, they also find the Firewalker, who wants to retain the power of his ancestors and put an end to the trio&#39;s treasure hunt. In the end, Max and his two friends persevere and return home wealthy

In [20]:
last_page['data'].keys()

dict_keys(['movie_count', 'limit', 'page_number', 'movies'])

In [29]:
final_page = 0
for i in range(821, 900, 1):
    last_page =  requests.get(f'https://yts.torrentbay.to/api/v2/list_movies.json?page={i}&limit=50').json()
    if 'movies' not in last_page['data'].keys():
        final_page = i-1
        break

In [28]:
last_page['data']['movies'][-1]

{'id': 1,
 'url': 'https://yts.torrentbay.to/movies/bikini-model-academy-2015',
 'imdb_code': 'tt3208802',
 'title': 'Bikini Model Academy',
 'title_english': 'Bikini Model Academy',
 'title_long': 'Bikini Model Academy (2015)',
 'slug': 'bikini-model-academy-2015',
 'year': 2015,
 'rating': 2.3,
 'runtime': 84,
 'genres': ['Action', 'Comedy'],
 'summary': 'When T. J. and Benji, two California twenty-something best buddies, lose their girlfriends, they start a home grown bikini modeling academy to make money and meet new girls. With a little help from T.J.&#39;s Uncle Seymour (Gary Busey), the guys begin recruiting pretty girls, until a rival modeling school owned by their old grade school enemy tries to shut them down.',
 'description_full': 'When T. J. and Benji, two California twenty-something best buddies, lose their girlfriends, they start a home grown bikini modeling academy to make money and meet new girls. With a little help from T.J.&#39;s Uncle Seymour (Gary Busey), the guys 

In [51]:
last_page =  requests.get('https://yts.torrentbay.to/api/v2/list_movies.json?page=1&limit=50').json()


In [23]:
min(l1)

1

In [46]:
ids = [41763,41713]
ids

[41763, 41713]

In [64]:
def check_new_movies(ids:list):

    page_number = 1
    movies_list = []
    while True:
        init_page = requests.get(f'https://yts.torrentbay.to/api/v2/list_movies.json?page={page_number}&limit=50').json()

        current_ids = [init_page['data']['movies'][0]['id'],init_page['data']['movies'][-1]['id']]

        if current_ids > ids:
            for movie in init_page['data']['movies']:
                movies_list.append(movie['title'])
        
            page_number = page_number + 1

        else:
            break

    return movies_list

In [65]:
mov_list = check_new_movies(ids)

1150

[41763, 41713]